In [33]:
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler

In [35]:
# Mount Google Drive
# from google.colab import drive
# drive.mount('/content/drive')

# Import necessary libraries
import os
import numpy as np
import librosa
from scipy.signal import lfilter
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout

# Function to extract LPC coefficients from waveform
def extract_lpc_coeffs(waveform, lpc_order):
    autocorr_coeffs = librosa.autocorrelate(waveform, max_size=lpc_order+1)
    lpc_coeffs = np.squeeze(lfilter([0] + -1 * autocorr_coeffs[1:].tolist(), [1], np.eye(lpc_order + 1), axis=0))
    return lpc_coeffs

# Load the dataset
data_dir = '/Users/khan/Downloads/archive/recording/'  # Update with the path to your dataset folder

X = []
y = []

# Iterate over files in the dataset directory
for file_name in os.listdir(data_dir):
    file_path = os.path.join(data_dir, file_name)
    waveform, sr = librosa.load(file_path, sr=None)
    lpc_order = 12
    lpc_coeffs = extract_lpc_coeffs(waveform, lpc_order)

    X.append(lpc_coeffs)
    # Extract the digit label from the first character of the filename
    label = int(file_name[0])
    y.append(label)

# Convert data to numpy arrays
X = np.array(X)
y = np.array(y)

# Perform label encoding
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

enc = OneHotEncoder(handle_unknown='ignore')
y_encoded=enc.fit_transform(y_encoded.reshape(-1,1)).toarray()

In [25]:
print(X.shape)
print(y_encoded.shape)

(3000, 13, 13)
(3000, 10)


In [41]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

X = X.reshape(-1,X.shape[0]*X.shape[1])

In [42]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(2400, 169)
(600, 169)
(2400, 10)
(600, 10)


In [85]:


# Ensure consistent number of samples
assert X_train.shape[0] == y_train.shape[0], "Number of samples in X_train and y_train do not match"
assert X_test.shape[0] == y_test.shape[0], "Number of samples in X_test and y_test do not match"

# Build the ANN model
model = Sequential()
model.add(Dense(256, activation='relu', input_dim=X_train.shape[1]))
model.add(Dense(128, activation='relu'))
model.add(Dense(len(np.unique(y)), activation='sigmoid'))  # Output layer with units equal to the number of unique labels

#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.compile(loss='binary_crossentropy', optimizer=RMSprop(learning_rate=0.001), metrics=['accuracy'])



# Train the model
model.fit(X_train, y_train, epochs=25, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss:.4f}')
print(f'Test Accuracy: {accuracy:.4f}')

Epoch 1/100
75/75 [==============================] - 1s 5ms/step - loss: 0.6756 - accuracy: 0.0975 - val_loss: 0.6608 - val_accuracy: 0.1200
Epoch 2/100
75/75 [==============================] - 0s 3ms/step - loss: 0.6472 - accuracy: 0.1000 - val_loss: 0.6335 - val_accuracy: 0.0850
Epoch 3/100
75/75 [==============================] - 0s 3ms/step - loss: 0.6206 - accuracy: 0.0942 - val_loss: 0.6076 - val_accuracy: 0.0917
Epoch 4/100
75/75 [==============================] - 0s 3ms/step - loss: 0.5954 - accuracy: 0.0992 - val_loss: 0.5831 - val_accuracy: 0.1067
Epoch 5/100
75/75 [==============================] - 0s 3ms/step - loss: 0.5715 - accuracy: 0.0967 - val_loss: 0.5599 - val_accuracy: 0.0850
Epoch 6/100
75/75 [==============================] - 0s 3ms/step - loss: 0.5490 - accuracy: 0.0988 - val_loss: 0.5381 - val_accuracy: 0.0883
Epoch 7/100
75/75 [==============================] - 0s 4ms/step - loss: 0.5278 - accuracy: 0.1008 - val_loss: 0.5176 - val_accuracy: 0.0850
Epoch 8/100
7